In [1]:
!pip install PyPDF2 transformers sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.1 MB/s eta 0:00:00


In [4]:
import PyPDF2
from transformers import pipeline
from google.colab import files

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text


In [5]:

import PyPDF2
from transformers import pipeline
from google.colab import files


def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text


In [6]:

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
qa_model = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

def summarize_cv(cv_text):
    chunk_size = 1000
    chunks = [cv_text[i:i+chunk_size] for i in range(0, len(cv_text), chunk_size)]

    summaries = []
    for chunk in chunks:
        summary = summarizer(chunk, max_length=130, min_length=30, do_sample=False)
        summaries.append(summary[0]['summary_text'])

    return " ".join(summaries)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu
Device set to use cpu


In [7]:



def answer_question(cv_text, question):
    result = qa_model(question=question, context=cv_text)
    return result['answer']

cv_text = extract_text_from_pdf(pdf_path)

print("📌 Original CV length:", len(cv_text), "characters")

summary = summarize_cv(cv_text)
print("\n📑 CV Summary:\n", summary)

questions = [
    "What skills does Mohamed Salama have?",
    "Where did Mohamed study?",
    "What projects has he worked on?"
]

print("\n🔎 Questions & Answers:")
for q in questions:
    print(f"Q: {q}")
    print("A:", answer_question(cv_text, q))


📌 Original CV length: 6894 characters

📑 CV Summary:
 Mohamed Salama is an AI Engineer with a solid foundation in Computer Science and a passion for building impactful, intelligent solutionsacross real-world applications. He has worked on projects in medical image analysis (anemia detection), segmentation, and NLP pipelines. Microsoft Machine Learning Smart Village, Giza. Includes Python basics, object-oriented programming, and essential libraries like NumPy, Pandas, and Matplotlib. Covers Python basics, object-oriented programming, and essential libraries like NumPy, Pandas, and Matplotlib. Includes math concepts like statistics, linear algebra, and data structures. C++ Programming Object-Oriented Concepts Covers C++ fundamentals, syntax, and advanced programming concepts. Data Structures Algorithms Covers essential data structures (arrays, linked lists, stacks, queues, trees, graphs) Built a sentiment classification system on 53,000+ mental health-related user statements using models

In [8]:
print(answer_question(cv_text, "Which university did he attend?"))


Ahram Canadian University


In [9]:
print(answer_question(cv_text, "what projects he do"))


AI, computer vision, Machine learning(ML), Deep Learning


In [11]:

import gradio as gr

def process(question):
    if question.strip() == "":
        return summarize_cv(cv_text)
    else:
        return answer_question(cv_text, question)

demo = gr.Interface(
    fn=process,
    inputs=gr.Textbox(label="Ask a Question (leave empty for summary)"),
    outputs="text",
    title="📄Q&A",
    description="Upload your CV PDF (already uploaded) and type a question, or leave empty to get a summary."
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9b2e0775b10c169b72.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
